In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from dataset import *
from models import ResNet18, ResNet50
import medmnist
from medmnist import INFO, Evaluator
from medmnist import BloodMNIST

import os
from collections import OrderedDict
from copy import deepcopy

import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet18, resnet34, resnet50
from torchvision.models import swin_v2_t


In [2]:
data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)


# TORCH 
# model = swin_v2_t(pretrained=True)
# model.head.out_features = n_classes
# num_ftrs = model.head.in_features
# model.head = nn.Linear(num_ftrs, n_classes)
# model.name = 'swinv2_t'

# model = resnet18(pretrained=True)
# model.fc.out_features = n_classes
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, n_classes)
# model.name = 'resnet18'

model = resnet34(pretrained=True)
model.fc.out_features = n_classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, n_classes)
model.name = 'resnet34'

train_2d_model(model, DEVICE, data_flag, milestones, gamma, output_root, task, train_loader, train_loader_at_eval, val_loader, test_loader, NUM_EPOCHS, lr, True)


# model = resnet50(pretrained=True)
# model.fc.out_features = n_classes
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, n_classes)
# model.name = 'resnet50'


# OWN
# model = ResNet18(in_channels=n_channels, num_classes=n_classes)

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
train:  82%|████████▏ | 77/94 [00:09<00:02,  8.18it/s]


KeyboardInterrupt: 

In [3]:
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}

Epoch 0 of 30


train: 100%|██████████| 94/94 [00:41<00:00,  2.27it/s]


Done with batches
Epoch loss 0.4581768531748589


test: 100%|██████████| 14/14 [00:02<00:00,  5.74it/s]


cur_best_auc: 0.9840345281461182
cur_best_epoch 0
Epoch 1 of 30


...

test: 100%|██████████| 14/14 [00:02<00:00,  6.15it/s]


Epoch 29 of 30


train: 100%|██████████| 94/94 [00:43<00:00,  2.17it/s]


Done with batches
Epoch loss 0.0010725041566787541


test: 100%|██████████| 14/14 [00:02<00:00,  6.37it/s]


In [4]:

path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

test: 100%|██████████| 14/14 [00:01<00:00,  7.26it/s]

bloodmnist
train  auc: 1.00000  acc: 1.00000
val  auc: 0.99827  acc: 0.96262
test  auc: 0.99765  acc: 0.96054



In [5]:
data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

model = resnet18(pretrained=True)
model.fc.out_features = n_classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, n_classes)
model.name = 'resnet18'



Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Epoch 0 of 30


train: 100%|██████████| 94/94 [00:24<00:00,  3.91it/s]


Done with batches
Epoch loss 0.4344546544425031


test: 100%|██████████| 14/14 [00:01<00:00,  8.82it/s]


cur_best_auc: 0.9848569441644346
cur_best_epoch 0
Epoch 1 of 30


...

Epoch 29 of 30


train: 100%|██████████| 94/94 [00:19<00:00,  4.74it/s]


Done with batches
Epoch loss 0.0019709859222162545


test: 100%|██████████| 14/14 [00:01<00:00,  9.74it/s]


bloodmnist
train  auc: 1.00000  acc: 0.99992
val  auc: 0.99854  acc: 0.95678
test  auc: 0.99763  acc: 0.96083



In [7]:

data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

model = resnet50(pretrained=True)
model.fc.out_features = n_classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, n_classes)
model.name = 'resnet50'

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Epoch 0 of 30


train: 100%|██████████| 94/94 [01:07<00:00,  1.40it/s]


Done with batches
Epoch loss 0.4749878576778351


test: 100%|██████████| 14/14 [00:04<00:00,  2.95it/s]


cur_best_auc: 0.9904587566110492
cur_best_epoch 0
Epoch 1 of 30


...

Epoch 29 of 30


train: 100%|██████████| 94/94 [01:24<00:00,  1.12it/s]


Done with batches
Epoch loss 0.00043391293033021207


test: 100%|██████████| 14/14 [00:06<00:00,  2.02it/s]


bloodmnist
train  auc: 1.00000  acc: 1.00000
val  auc: 0.99823  acc: 0.96262
test  auc: 0.99781  acc: 0.96405



In [21]:

data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

from torchvision.models import convnext_small

model = convnext_small(weights=True)
print(model.classifier)
model.classifier[2] = nn.Linear(768, n_classes)
model.name = 'convnext_small'

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): LayerNorm2d((768,), eps=1e-06, elementwise_affine=True)
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=768, out_features=1000, bias=True)
)
Epoch 0 of 30


train:   0%|          | 0/94 [00:00<?, ?it/s]


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (2 x 2). Kernel size can't be greater than actual input size

In [13]:

data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

from torchvision.models import vit_b_16

model = vit_b_16(pretrained=True)
model.heads = nn.Linear(768, n_classes)
model.name = 'vit_b_16'

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 of 30


train:   0%|          | 0/94 [00:00<?, ?it/s]


AssertionError: Wrong image height! Expected 224 but got 28!

In [22]:

data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

from torchvision.models import efficientnet_v2_s

model = efficientnet_v2_s(pretrained=True)
model.classifier[1] = nn.Linear(1280, n_classes)
model.name = 'efficientnet_v2_s'

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 of 30


train: 100%|██████████| 94/94 [03:08<00:00,  2.01s/it]


Done with batches
Epoch loss 0.699226622885846


...

cur_best_auc: 0.9980650900255572
cur_best_epoch 28
Epoch 29 of 30


train: 100%|██████████| 94/94 [02:54<00:00,  1.86s/it]


Done with batches
Epoch loss 0.0017613798560036076


test: 100%|██████████| 14/14 [00:07<00:00,  1.79it/s]

bloodmnist
train  auc: 1.00000  acc: 1.00000
val  auc: 0.99807  acc: 0.96028
test  auc: 0.99770  acc: 0.96141



In [19]:

data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

from torchvision.models import inception_v3

model = inception_v3(pretrained=True)
model.fc = nn.Linear(2048, n_classes)
model.name = 'inception_v3'


model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Linear(in_features=2048, out_features=1000, bias=True)
Epoch 0 of 30


train:   0%|          | 0/94 [00:00<?, ?it/s]


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [23]:
data_flag = 'bloodmnist'
download = True

DEVICE = 'mps'
NUM_EPOCHS = 30
BATCH_SIZE = 128
milestones = [0.5 * NUM_EPOCHS, 0.75 * NUM_EPOCHS]
lr = 0.001
gamma = 0.1

output_root = './output2d'

info = INFO[data_flag]
n_channels = info['n_channels']
n_classes = len(info['label'])
task = info['task']

train_loader, train_loader_at_eval, val_loader, test_loader = extract_data(data_flag, download, BATCH_SIZE)

from torchvision.models import swin_v2_s

model = swin_v2_s(pretrained=True)
model.head = nn.Linear(768, n_classes)
model.name = 'swin_v2_s'

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

train_evaluator = medmnist.Evaluator(data_flag, 'train')
val_evaluator = medmnist.Evaluator(data_flag, 'val')
test_evaluator = medmnist.Evaluator(data_flag, 'test')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

logs = ['loss', 'auc', 'acc']
train_logs = ['train_'+log for log in logs]
val_logs = ['val_'+log for log in logs]
test_logs = ['test_'+log for log in logs]
log_dict = OrderedDict.fromkeys(train_logs+val_logs+test_logs, 0)

writer = SummaryWriter(log_dir=os.path.join(output_root, model.name))

best_auc = 0
best_epoch = 0
best_model = deepcopy(model)
tb_twod = True

for epoch in range(NUM_EPOCHS):        
    print(f"Epoch {epoch} of {NUM_EPOCHS}")
    train_loss = train(model, train_loader, task, criterion, optimizer, DEVICE, writer)
    
    train_metrics = test(model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model1')
    val_metrics = test(model, val_evaluator, val_loader, task, criterion, DEVICE, 'model1')
    test_metrics = test(model, test_evaluator, test_loader, task, criterion, DEVICE, 'model1')
    
    scheduler.step()
    
    for i, key in enumerate(train_logs):
        log_dict[key] = train_metrics[i]
    for i, key in enumerate(val_logs):
        log_dict[key] = val_metrics[i]
    for i, key in enumerate(test_logs):
        log_dict[key] = test_metrics[i]

    for key, value in log_dict.items():
        writer.add_scalar(key, value, epoch)
        
    cur_auc = val_metrics[1]
    if cur_auc > best_auc:
        best_epoch = epoch
        best_auc = cur_auc
        best_model = deepcopy(model)
        print('cur_best_auc:', best_auc)
        print('cur_best_epoch', best_epoch)

state = {
    'net': best_model.state_dict(),
}


path = os.path.join(output_root, f'{model.name}_best_model.pth')
torch.save(state, path)

train_metrics = test(best_model, train_evaluator, train_loader_at_eval, task, criterion, DEVICE, 'model', output_root)
val_metrics = test(best_model, val_evaluator, val_loader, task, criterion, DEVICE, 'model', output_root)
test_metrics = test(best_model, test_evaluator, test_loader, task, criterion, DEVICE, 'model', output_root)

train_log = 'train  auc: %.5f  acc: %.5f\n' % (train_metrics[1], train_metrics[2])
val_log = 'val  auc: %.5f  acc: %.5f\n' % (val_metrics[1], val_metrics[2])
test_log = 'test  auc: %.5f  acc: %.5f\n' % (test_metrics[1], test_metrics[2])

log = '%s\n' % (data_flag) + train_log + val_log + test_log
print(log)
        
with open(os.path.join(output_root, '%s_log.txt' % (data_flag)), 'a') as f:
    f.write(log)  

writer.close()

Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz
Using downloaded and verified file: /Users/vemundlund/.medmnist/bloodmnist.npz


/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vemundlund/miniconda3/envs/vi_venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=Swin_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 of 30


train: 100%|██████████| 94/94 [10:21<00:00,  6.61s/it]


Done with batches
Epoch loss 1.8202674820068034


...

train: 100%|██████████| 94/94 [10:10<00:00,  6.49s/it]


Done with batches
Epoch loss 0.17180371403377107


test: 100%|██████████| 14/14 [00:39<00:00,  2.79s/it]


cur_best_auc: 0.9926174590649204
cur_best_epoch 17
Epoch 18 of 30


train:  13%|█▎        | 12/94 [13:17<58:29, 42.80s/it]  

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir 'output2d'
from tensorboard import notebook
notebook.list()


In [10]:
# train  auc: 0.99237  acc: 0.90894
# val  auc: 0.99129  acc: 0.90362
# test  auc: 0.99095  acc: 0.89711